<a href="https://colab.research.google.com/github/hoju-spe/gachon-ml-2025/blob/main/RAG%EC%A0%81%EC%9A%A9_%EA%B8%B0%EA%B3%84%ED%94%84%EC%B5%9C%EC%B5%9C%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
#  300개 학습데이터 자동 생성기 (JSONL)
# ==========================================
from openai import OpenAI
import json
import os
os.environ["OPENAI_API_KEY"] = "키"
client = OpenAI()

def generate_sample():
    prompt = """
다음 형식의 면접 학습 데이터를 하나 생성하세요.

형식(JSON):
{
 "instruction": "면접 질문",
 "input": "원본 답변",
 "output": "STAR 기반 개선 답변"
}

조건:
- 질문은 실제 면접에서 자주 나오는 질문일 것
- 원본 답변은 ‘부족한/짧은/구체적이지 않은’ 답변일 것
- output 은 원본 답변만을 기반으로 STAR 구조로 확장
- output 에 새로운 경험을 창작하지 말 것
"""

    res = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.6
    )
    txt = res.choices[0].message.content.strip()
    return json.loads(txt)

dataset = []
for i in range(300):
    print(f"{i+1}/300 생성중...")
    try:
        item = generate_sample()
        dataset.append(item)
    except:
        continue

with open("training_data.jsonl", "w", encoding="utf-8") as f:
    for d in dataset:
        f.write(json.dumps(d, ensure_ascii=False) + "\n")

print(" training_data.jsonl 생성 완료!")

In [ ]:
!pip install chromadb -q


In [ ]:
import json
import chromadb
from chromadb.config import Settings

# Chroma 클라이언트 생성
chroma_client = chromadb.PersistentClient(path="/content/chroma_db")

collection = chroma_client.get_or_create_collection(
    name="interview_data",
    metadata={"hnsw:space": "cosine"}
)
# collection 생성
collection = chroma_client.get_or_create_collection(
    name="interview_data",
    metadata={"hnsw:space": "cosine"}
)

# JSONL 로드
data = []
with open("/content/training_data.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

# DB에 넣기
documents = []
metadatas = []
ids = []

for i, item in enumerate(data):
    text = f"""
[면접 질문] {item['instruction']}
[짧은 답변] {item['input']}
[STAR 확장 답변] {item['output']}
"""
    documents.append(text)
    metadatas.append({"category": "interview"})
    ids.append(str(i))

collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

print("300개 데이터 ChromaDB 저장 완료!")


🔥 300개 데이터 ChromaDB 저장 완료!


In [ ]:
def retrieve_similar_answers(query, top_k=3):
    results = collection.query(
        query_texts=[query],
        n_results=top_k
    )

    retrieved = []
    for doc in results["documents"][0]:
        retrieved.append(doc)

    return "\n\n---\n\n".join(retrieved)


In [ ]:
# 1. 필수 라이브러리 설치
!pip install gradio openai transformers accelerate bitsandbytes chromadb -q

import gradio as gr
from openai import OpenAI
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
import chromadb
import torch
import os


# ==========================================
#  OpenAI API 키
# ==========================================
os.environ["OPENAI_API_KEY"] = "키"
client = OpenAI()


# ==========================================
#  ChromaDB (RAG) 초기화
# ==========================================
chroma_client = chromadb.PersistentClient(path="/content/chroma_db")

collection = chroma_client.get_or_create_collection(
    name="interview_data",
    metadata={"hnsw:space": "cosine"}
)


# ==========================================
#  Qwen 2.5-3B-Instruct 로드 (감정 분석용)
# ==========================================
MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"

print(" Qwen 2.5 3B 모델 로딩 중... (4bit 양자화)")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True
)

# 토크나이저 로드 부분
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=quant_config,
    device_map={"": "cuda"}  # GPU에 강제로 올림
)
print(" 모델 로딩 완료!")


# ==========================================
#  RAG 검색 함수
# ==========================================
def retrieve_similar_answers(query, top_k=3):
    try:
        results = collection.query(
            query_texts=[query],
            n_results=top_k
        )
        docs = results.get("documents", [[]])[0]
        return "\n\n---\n\n".join(docs)
    except:
        return "(RAG 검색 실패, DB 비어 있음)"


# ==========================================
#  감정 분석 (Qwen 2.5 3B 로컬 LLM)
# ==========================================
def analyze_sentiment_local(text):
    # 1. 시스템 프롬프트로 강하게 제약 (사족 금지, 문장 완성)
    system_message = """당신은 냉철한 면접관입니다. 지원자의 답변을 분석하세요.
규칙:
1. '정보가 부족하다'거나 '추측한다'는 말은 절대 하지 마세요.
2. 주어진 텍스트만으로 판단하세요.
3. 각 항목의 문장은 반드시 완결된 문장으로 끝내세요.
4. 아래 형식을 엄격히 지키세요.

형식:
tone: (감정 톤)
confidence: (자신감 수준)
positive: (긍정적인 점)
negative: (아쉬운 점)
summary: (총평)"""

    user_message = f"답변: {text}\n\n위 답변을 분석해."

    # Qwen의 채팅 템플릿 적용 (지시 이행력 상승)
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    text_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text_input], return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,  # 200 -> 512 (문장 끊김 방지)
            do_sample=True,
            top_p=0.9,
            temperature=0.3,
            repetition_penalty=1.1 # 반복 방지
        )

    # 입력 프롬프트 부분 잘라내기
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response.strip()



# ==========================================
#  STAR 확장 (GPT-4o-mini + RAG)
# ==========================================
def improve_answer_star(question, answer):
    rag_context = retrieve_similar_answers(question)

    system_prompt = "당신은 STAR/PMI 기반의 전문 면접 코치입니다."

    user_prompt = f"""
면접 질문: {question}
사용자 답변: {answer}

[RAG 참고 사례]
{rag_context}

아래 조건에 맞게 면접 답변을 STAR 구조로 자연스럽게 확장하세요.

조건:
- 경험 창작 금지 (사용자 답변 범위 내에서 확장)
- STAR(상황-과제-행동-결과) 구조 유지
- 문장 수 6~10개
- 면접관이 듣기 좋은 현실적인 방식으로 작성
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content



# ==========================================
#  통합 함수
# ==========================================
def run_all(question, answer):
    if not question or not answer:
        return "질문과 답변을 입력해주세요.", "입력 대기 중..."

    improved = improve_answer_star(question, answer)     # GPT-4o-mini
    sentiment = analyze_sentiment_local(answer)          # Qwen 2.5 3B
    return improved, sentiment



# ==========================================
#  Gradio UI
# ==========================================
with gr.Blocks(title="AI 면접 스피치 코치 PICO") as demo:

    gr.Markdown("# 🎤 AI 면접 스피치 코치 PICO")

    with gr.Row():
        with gr.Column(scale=1):
            q_input = gr.Textbox(label="면접 질문", lines=2, placeholder="예: 본인의 강점은 무엇인가요?")
            a_input = gr.Textbox(label="나의 답변", lines=10, placeholder="답변을 입력하세요.")
            btn = gr.Button("✨ 분석 및 첨삭 받기", variant="primary")

        with gr.Column(scale=1):
            a_output = gr.Textbox(label="AI 수정 답변 (STAR)", lines=12)
            s_output = gr.Textbox(label="면접관 심리/감정 분석", lines=12)

    btn.click(
        fn=run_all,
        inputs=[q_input, a_input],
        outputs=[a_output, s_output]
    )

print("🌐 Web UI 실행 중…")
demo.launch(share=True, debug=True)

🚀 Qwen 2.5 3B 모델 로딩 중... (4bit 양자화)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 모델 로딩 완료!
🌐 Web UI 실행 중…
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://db29d493283cf21229.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://db29d493283cf21229.gradio.live
